# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0162, -0.1235,  0.0978, -0.1359,  0.0353,  0.0444,  0.0626,  0.2392,
         -0.1748,  0.2616],
        [-0.0665, -0.0984, -0.0934, -0.1049,  0.0151,  0.0801, -0.0665,  0.1619,
         -0.2138,  0.1742]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.1847,  0.1681, -0.0677, -0.0248,  0.0628, -0.1342, -0.0590,  0.2103,
         -0.1344, -0.0614],
        [-0.0814,  0.1199,  0.0536, -0.0940,  0.0298, -0.1175, -0.2023,  0.1251,
         -0.0909, -0.0490]], grad_fn=<AddmmBackward0>)

顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2714, -0.0470, -0.0670,  0.3418, -0.0940,  0.0299,  0.0071, -0.0880,
         -0.3707,  0.2247],
        [-0.2689,  0.0794, -0.0820,  0.2196, -0.0927, -0.0532,  0.0645, -0.0502,
         -0.1984,  0.0872]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0073, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3243, grad_fn=<SumBackward0>)